<a href="https://colab.research.google.com/github/thadeupenna/visualizacao/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acompanhamento de casos da COVID-19

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import Request, urlopen
import io
import gzip

## Baixa os dados

In [ ]:
url = "https://data.brasil.io/dataset/covid19/caso_full.csv.gz"

request = Request(url, headers={"User-Agent": "python-urllib"})
response = urlopen(request)
data = gzip.decompress(response.read()).decode("utf-8")
df = pd.read_csv(io.StringIO(data))

In [ ]:
df.head()

In [ ]:
df.info()

## Dados de entrada


In [ ]:
municipio = 'Volta Redonda'
estado = 'RJ'

Criando um dataframe que é uma "fatia" do original. 

In [ ]:
municipio_df = df[(df['city'] == municipio) & (df['state'] == estado)]

In [ ]:
municipio_df.head()

Transformando a coluna `date` de `str` para `datetime`

In [ ]:
municipio_df.date = pd.to_datetime(municipio_df.date)

### SettingWithCopyWarning: 

```
SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
```






In [ ]:
municipio_df.info()

In [ ]:
municipio_df.loc[:,'date'] = pd.to_datetime(municipio_df.date)

Explicamos ao Pandas que, explicitamente, queremos uma cópia.

In [ ]:
municipio_df = df[(df['city'] == municipio) & (df['state'] == estado)].copy()

In [ ]:
municipio_df['date'] = pd.to_datetime(municipio_df.date)

In [ ]:
municipio_df.info()

### Por que usar `datetime` ? 

Documentação em [https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt)

In [ ]:
municipio_df.date.dt.dayofyear.iloc[-1] -  municipio_df.date.dt.dayofyear.iloc[0]

In [ ]:
municipio_df.date.max() - municipio_df.date.min()

In [ ]:
municipio_df.columns

In [ ]:
municipio_index = pd.DatetimeIndex(municipio_df.date.values)

In [ ]:
municipio_df.set_index(municipio_index, inplace=True)

In [ ]:
municipio_df.head()

In [ ]:
municipio_df.index.name = 'day'

In [ ]:
municipio_df.head()

In [ ]:
municipio_df.drop('date',axis=1, inplace=True)

In [ ]:
municipio_df.head()

In [ ]:
plt.figure(figsize=(10,8))

plt.bar(municipio_df.index, municipio_df.last_available_confirmed, label= 'Contaminados')
plt.legend()

# Múltiplos Plots

[texto do link](https://)Plotar, para um dado município, os casos confirmados, os óbitos, e os  casos e óbitos diários, como uma matriz 2x2. 

Estrutura do `Matplotlib`

![texto alternativo](https://matplotlib.org/_images/anatomy.png)

Vamos ter um Figure e 4 Axes 

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(18,10), facecolor='lightgrey')
type(fig)
type(axs)


Rodando em Python, você precisaria adicionar o `plt.show()`. No Jupyter, usa-se `%matplotlib inline` ou `%matplotlib notebook`

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(18,10), facecolor='lightgrey')

x = municipio_df.index
axs[0, 0].bar(x, municipio_df.last_available_confirmed, label = 'Casos Confirmados')
axs[0, 1].bar(x, municipio_df.new_confirmed, label = 'Novos Casos')
axs[1, 0].bar(x, municipio_df.last_available_deaths, label = 'Óbitos', color='red')
axs[1, 1].bar(x, municipio_df.new_deaths, label = 'Novos Óbitos', color = 'red')
axs[0,0].legend()
axs[0,1].legend()
axs[1,0].legend()
axs[1,1].legend()
plt.xticks(x[::7], rotation=45)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(20,10), facecolor='darkgrey', constrained_layout=True, sharex=True)
fig.suptitle("COVID-19 em "+municipio, fontsize=24)
x = municipio_df.index.strftime("%d/%m")
axs[0, 0].bar(x, municipio_df.last_available_confirmed, label = 'Casos Confirmados')
axs[0, 1].bar(x, municipio_df.new_confirmed, label = 'Novos Casos')
axs[1, 0].bar(x, municipio_df.last_available_deaths, label = 'Óbitos', color='red')
axs[1, 1].bar(x, municipio_df.new_deaths, label = 'Novos Óbitos', color = 'red')

for a1 in range(len(axs)):
  for a2 in range(len(axs[a1])):
    axs[a1, a2].legend(frameon=False, loc='upper left')
    axs[a1,a2].xaxis.set_major_locator(plt.MultipleLocator(14))
    axs[a1,a2].xaxis.set_minor_locator(plt.MultipleLocator(7))
    axs[a1,a2].grid(b=True, which='both', color='black', linestyle=':')
    axs[a1,a2].set_facecolor('lightgrey')
    axs[a1,a2].spines['right'].set_visible(False)
    axs[a1,a2].spines['top'].set_visible(False)
    axs[a1,a2].spines['left'].set_visible(False)
    axs[a1,a2].spines['bottom'].set_visible(False)

# plt.tight_layout()

# Intervalo de interesse

In [ ]:
ultimo_mes = municipio_df.loc[municipio_df.index>='2020-07-01'].copy()

In [ ]:
inicio = pd.to_datetime('07/01/2020')
inicio

In [ ]:
municipio_index[0]

In [ ]:
ultimo_mes.head()

In [ ]:
ultimo_mes.info()

# Dicas de análise

1. Análise por estado. Verifique a linha de filtro. Pode-se adicionar um `if`, caso `municipio` seja ' ' 
2. Dados do Brasil inteiro. `groupy(estado).sum()`. Pode-se adicionar outro `if`
``` 
if estado == '' :
  # cria uma planilha com os dados agrupados 
  local_df = .... 
elif municipio == '':
  # Nenhum municipio mas selecionou estado    
  local_df = df[df[estado] == ] 
else : 
  local_df = df[df[municipio] == municipio....
```
E repete as instruções com `local`no lugar de `municipio`
3. Escolher as 10 cidades mais atingidas em cada período e acompanhar, em um gráfico de linha, em que posição estavam em cada dia. Exemplo: 
![](http://theroamingpint.com/wp-content/uploads/2013/04/Top-10-Breweries-over-time.png)

# Como enviar o notebook ...

1.   Limpem as instruções intermediárias (head()s, info()s)
2.   Em Ambiente de Execução, Reiniciar e executar tudo.
3.   Salvar (Ctrl+S)
4.   Confira o que foi gravado e compartilhe
